# Automating NVMe Drive Mounting on EC2 with a .bat Script


In this notebook, we provide a step-by-step guide to create a `.bat` script for automating the process of mounting an ephemeral NVMe SSD drive on an EC2 Windows Server instance.

Every time the EC2 instance stops, ephemeral NVMe storage is unmounted. This `.bat` script, when run, will re-partition, format, and assign a drive letter to the NVMe volume, making it available for use.



## Prerequisites
- **Administrator Privileges**: You need to run the `.bat` file with administrator privileges for it to execute successfully.
- **Drive Verification**: Identify the disk number of your NVMe drive using `Disk Management` in Windows. Note this disk number as you will use it in the script.



## Step 1: Create the `.bat` File
Below is the `.bat` script to automate partitioning, formatting, and assigning a drive letter to your NVMe volume.

### Script

Save the following script as `mount_nvme_drive.bat`.

```bat
@echo off
:: Set the disk number and drive letter
set DISK_NUMBER=1
set DRIVE_LETTER=Z

:: Check if the drive letter is already assigned
wmic logicaldisk get name | find "%DRIVE_LETTER%:" >nul
if %errorlevel% equ 0 (
    echo Drive %DRIVE_LETTER% is already mounted.
    exit /b
)

:: Create DiskPart script file dynamically
echo list disk > "%temp%\diskpart_script.txt"
echo select disk %DISK_NUMBER% >> "%temp%\diskpart_script.txt"
echo clean >> "%temp%\diskpart_script.txt"
echo create partition primary >> "%temp%\diskpart_script.txt"
echo format fs=ntfs quick >> "%temp%\diskpart_script.txt"
echo assign letter=%DRIVE_LETTER% >> "%temp%\diskpart_script.txt"

:: Run DiskPart with the script
diskpart /s "%temp%\diskpart_script.txt"

:: Clean up the script file
del "%temp%\diskpart_script.txt"

echo Drive %DRIVE_LETTER% has been mounted and formatted.
pause
```

### Explanation

1. **Set Disk Number and Drive Letter**: Replace `DISK_NUMBER` with your NVMe drive's disk number (default is `1`). Choose an available `DRIVE_LETTER`, like `Z`.

2. **Create DiskPart Script**: The script dynamically creates a `diskpart_script.txt` file that DiskPart uses to process commands for partitioning, formatting, and assigning the drive letter.

3. **Clean, Partition, Format, and Assign Drive Letter**: The script uses DiskPart commands:
   - `clean`: Clears the disk, removing all existing data.
   - `create partition primary`: Creates a primary partition.
   - `format fs=ntfs quick`: Formats the partition as NTFS.
   - `assign letter=Z`: Assigns drive letter `Z` (or the specified letter).



## Important Notes

- **Disk Number**: Make sure `DISK_NUMBER=1` in the script matches your actual NVMe disk. Adjust it if the number is different.
- **Drive Letter**: The drive letter `Z` is assigned; you can modify this to any available letter.
- **Data Loss Warning**: The `clean` command erases all existing data on the selected disk. Only use this on an ephemeral NVMe drive or if you're sure data loss is acceptable.



## Step 2: Running the Script

1. Save the `.bat` file with the provided content.
2. **Run the script as an administrator** by right-clicking and selecting "Run as Administrator."
3. Upon running, the script will mount the NVMe drive, format it if needed, and assign the specified drive letter.

You should see output indicating the drive's successful mounting and formatting. 

**Optional:** To automate mounting at startup, place the `.bat` file in the Windows Startup folder:
   - Press `Win + R`, type `shell:startup`, and press Enter.
   - Add the `.bat` file to this folder.
   
This will ensure the script runs every time the instance starts, preparing the NVMe drive for use.



## Summary

This notebook provides a `.bat` script that automates mounting an ephemeral NVMe SSD on a Windows Server EC2 instance. The script dynamically creates a DiskPart script to:
- Clean the NVMe disk (if needed).
- Create a partition, format it to NTFS, and assign a drive letter.

This script simplifies NVMe mounting and can be set to run at startup for convenience.
